# Загрузим и почистим хэштеги

In [ ]:
df = pd.read_csv("yappy_hackaton_2024_400k.csv")[["description"]]

In [ ]:
df.fillna(-100, inplace = True)
df = df[df["description"]!=-100]

In [ ]:
df["synonyms"] = df["description"].apply(lambda x: re.sub(r'[^a-zA-Zа-яА-Я0-9 #]', ' ', x).lower().split())

In [ ]:
res_syn=[]
for row in df.iterrows():
  res_temp = []
  syn = row[1]["synonyms"]
  for s in syn:
    if s.startswith("#"):
      res_temp.append(s[1:])
    else:
      continue
  res_temp = [re.sub(r'[^a-zA-Zа-яА-Я0-9]', ' ', x).lower().split() for x in res_temp]
  res_temp = [x for y in res_temp for x in y]
  res_syn.append(res_temp)

df["post_synonyms"] = res_syn

In [ ]:
df

,description,synonyms,post_synonyms
0,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m...","[#нарезкистримов, #dota2, #cs2, #fifa23, #mine...","[нарезкистримов, dota2, cs2, fifa23, minecraft..."
1,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...,"[не, вводи, эту, команду, в, роблокс, #shorts,...","[shorts, roblox, роблокс]"
2,"#boobs , #красивыедевушки , #ass","[#boobs, #красивыедевушки, #ass]","[boobs, красивыедевушки, ass]"
6,#diy #постановка #юмор #комедия,"[#diy, #постановка, #юмор, #комедия]","[diy, постановка, юмор, комедия]"
8,#образ #lookbook #показ #неделямоды #капсула #...,"[#образ, #lookbook, #показ, #неделямоды, #капс...","[образ, lookbook, показ, неделямоды, капсула, ..."
...,...,...,...
399995,#ceramics #handmade #ceramic #clay #ceramicscu...,"[#ceramics, #handmade, #ceramic, #clay, #ceram...","[ceramics, handmade, ceramic, clay, ceramicscu..."
399996,#красивыедевушки #красотки #грудь #boobs #секс...,"[#красивыедевушки, #красотки, #грудь, #boobs, ...","[красивыедевушки, красотки, грудь, boobs, секс..."
399997,"#лайфхаки , #эксперименты , #roblox , #игрушки...","[#лайфхаки, #эксперименты, #roblox, #игрушки, ...","[лайфхаки, эксперименты, roblox, игрушки, diy,..."
399998,#образ #lookbook #показ #неделямоды #капсула #...,"[#образ, #lookbook, #показ, #неделямоды, #капс...","[образ, lookbook, показ, неделямоды, капсула, ..."


# Посчитаем совместные совстречаемости слов в хэштегах

In [ ]:
def count_cooccurrences(df, column, n):
    cooccurrence_counter = Counter()

    for word_list in df[column]:
        for pair in combinations(sorted(word_list), n):
            cooccurrence_counter[pair] += 1

    cooccurrence_df = pd.DataFrame(
        cooccurrence_counter.items(),
        columns=['pair', 'count']
    )

    return cooccurrence_df

cooccurrence_2 = count_cooccurrences(df, 'post_synonyms', 2)

In [ ]:
cooccurrence_2

,pair,count
0,"(cs2, dota2)",7959
1,"(cs2, fifa23)",7959
2,"(cs2, genshin)",7957
3,"(cs2, minecraft)",7959
4,"(cs2, геншин)",7959
...,...,...
202236,"(gorkypark, часземли)",1
202237,"(паркгорького, часземли)",1
202238,"(bokunoheroacademia, edit)",1
202239,"(bokunoheroacademia, midoriya)",1


# Посмотрим на близость слов LaBSE

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/516M [00:00<?, ?B/s]

In [ ]:
def get_embeddings(sentences, batch_size=64):
    embeddings_list = []
    for i in tqdm(range(0, len(sentences), batch_size)):
        batch = list(sentences[i:i+batch_size])
        encoded_input = tokenizer(batch, padding=True, truncation=True, max_length=64, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)
        embeddings = model_output.pooler_output
        embeddings = torch.nn.functional.normalize(embeddings)
        embeddings_list.append(embeddings)

    all_embeddings = torch.cat(embeddings_list, dim=0)
    return all_embeddings

In [ ]:
sentences = list(df.explode("post_synonyms")["post_synonyms"].astype(str).unique())

In [ ]:
embeddings = get_embeddings(sentences)

  0%|          | 0/433 [00:00<?, ?it/s]

In [ ]:
cos_sim = cosine_similarity(embeddings.numpy())

In [ ]:
similarity_df = pd.DataFrame(cos_sim, index=sentences, columns=sentences)

In [ ]:
similarity_df.head()

,нарезкистримов,dota2,cs2,fifa23,minecraft,майнкрафт,геншин,genshin,shorts,roblox,...,шатландия,adite,mahiru,криш,встраненевыученныхуроков,экскаватор,southbea,камушки,часземли,midoriya
нарезкистримов,1.000000,0.246377,0.350015,0.319676,0.319242,0.351901,0.426403,0.393235,0.679369,0.417792,...,0.335238,0.301641,0.313909,0.441593,0.349059,0.365109,0.288282,0.453993,0.438647,0.318704
dota2,0.246377,1.000000,0.608631,0.585078,0.212704,0.237953,0.266656,0.300976,0.411615,0.385790,...,0.406241,0.524708,0.373346,0.403794,0.232548,0.299805,0.453353,0.338452,0.348106,0.464679
cs2,0.350015,0.608631,1.000000,0.487641,0.226280,0.229233,0.357553,0.340757,0.414423,0.254838,...,0.447336,0.368948,0.322452,0.399324,0.223484,0.306385,0.365089,0.275102,0.428620,0.357939
fifa23,0.319676,0.585078,0.487641,1.000000,0.266653,0.360697,0.357870,0.422673,0.411830,0.404959,...,0.238773,0.372509,0.443575,0.436557,0.205181,0.346002,0.397808,0.272048,0.269300,0.446595
minecraft,0.319242,0.212704,0.226280,0.266653,1.000000,0.644780,0.380997,0.359173,0.263303,0.317925,...,0.311686,0.258306,0.531002,0.360377,0.137519,0.505810,0.295891,0.500566,0.383309,0.416293
